# NYSE & Blurr

In this guide we will train a machine learning model that predicts closing price of a stock based on historical data. We will transform time-series stock data into features to train this model. 

## Prerequisites

It's recommended to have a basic understanding of how Blurr works. Following [tutorials 1](http://productml-blurr.readthedocs.io/en/latest/Streaming%20DTC%20Tutorial/) and [2](http://productml-blurr.readthedocs.io/en/latest/Window%20DTC%20Tutorial/) should provide enough background context.


## Preparation

Let's start by installing `Blurr` and other required dependencies:

In [1]:
import sys

print("installing blurr, keras and tensorflow...")
!{sys.executable} -m pip install blurr-dev --quiet
!{sys.executable} -m pip install keras --quiet
!{sys.executable} -m pip install tensorflow --quiet
print("done.")

installing blurr, keras and tensorflow...
done.


## The Dataset

This walkthrough is based on [New York Stock Exchange Data](https://www.kaggle.com/dgawlik/nyse/data) made available for [Kaggle challenges](https://www.kaggle.com/dgawlik/nyse).

Let's start by downloading and having a peek at the available data: 

In [2]:
!wget http://demo.productml.com/data/prices-split-adjusted.json.zip
!unzip -o prices-split-adjusted.json.zip -d .

--2018-05-02 14:45:06--  http://demo.productml.com/data/prices-split-adjusted.json.zip
Resolving demo.productml.com (demo.productml.com)... 52.218.201.147
Connecting to demo.productml.com (demo.productml.com)|52.218.201.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22162139 (21M) [application/zip]
Saving to: ‘prices-split-adjusted.json.zip.3’

prices-split-adjust 100%[===================>]  21.13M   776KB/s    in 23s     

2018-05-02 14:45:30 (930 KB/s) - ‘prices-split-adjusted.json.zip.3’ saved [22162139/22162139]

Archive:  prices-split-adjusted.json.zip
  inflating: ./prices-split-adjusted.json  


In [3]:
import pandas as pd

stocks = pd.read_json("./prices-split-adjusted.json", lines=True)
stocks.head()

,close,date,high,low,open,symbol,volume
0,125.839996,2016-01-05,126.250000,122.309998,123.430000,WLTW,2163600
1,119.980003,2016-01-06,125.540001,119.940002,125.239998,WLTW,2386400
2,114.949997,2016-01-07,119.739998,114.930000,116.379997,WLTW,2489500
3,116.620003,2016-01-08,117.440002,113.500000,115.480003,WLTW,2006300
4,114.970001,2016-01-11,117.330002,114.089996,117.010002,WLTW,1408600


This dataset contains data for each market day.

Our **goal is to predict closing price** of a stock for any given day based on historical data. In order to do that, we need to transform our original data source into **features** that can be used for training.

We'll calculate **moving averages** and other aggregate data for different **time windows**: one, three and seven days.

## Blurr Templates

We perform initial transformation of our data with [nyse-streaming-dtc.yml](./nyse-streaming-dtc.yml). Data is then aggregated by time using [nyse-window-dtc.yml](./nyse-window-dtc.yml).

In [4]:
!cat nyse-streaming-dtc.yml

Type: 'Blurr:Transform:Streaming'
Version: '2018-03-01'
Description: New York Store Exchange Transformations
Name: nyse

Import:
  - { Module: datetime, Identifiers: [ datetime ] }

Identity: source.symbol

Time: datetime.strptime(source.date, '%Y-%m-%d')

Stores:
  - Type: 'Blurr:Store:Memory'
    Name: memory

Aggregates:
  - Type: 'Blurr:Aggregate:Block'
    Name: stats
    Store: memory
    Split: True
    When: source.symbol in ['AAPL', 'MSFT', 'GOOG', 'FB']
    Fields:
      - Name: close
        Type: float
        Value: source.close

      - Name: volatility
        Type: float
        Value: (float(source.high) / float(source.low)) - 1

      - Name: volume
        Type: float
        Value: source.volume

**Streaming DTC**

We're predicting values for tech companies only (Apple, Facebook, Microsoft, Google):

```yaml
When: source.symbol in ['AAPL', 'MSFT', 'GOOG', 'FB']
```

Each record in the original dataset represents a single day, which is the same we need to feed our window transformation. By setting `Split: True` we'll create a new aggregate for each single dataset record.


In [5]:
!cat 'nyse-window-dtc.yml'

Type: Blurr:Transform:Window
Version: '2018-03-01'
Name: moving_averages

SourceDTC: nyse

Anchor:
  Condition: nyse.stats.volatility < 0.04

Aggregates:


  - Type: Blurr:Aggregate:Window
    Name: close
    WindowType: count
    WindowValue: 1
    Source: nyse.stats
    Fields:
    - Name: value
      Type: float
      Value: anchor.close

  - Type: Blurr:Aggregate:Window
    Name: last
    WindowType: count
    WindowValue: -1
    Source: nyse.stats
    Fields:
    - Name: close
      Type: float
      Value: source.close[0]
    - Name: volume
      Type: float
      Value: source.volume[0]
    - Name: volatility
      Type: float
      Value: source.volatility[0]

  - Type: Blurr:Aggregate:Window
    Name: last_3
    WindowType: count
    WindowValue: -3
    Source: nyse.stats
    Fields:
    - Name: close_avg
      Type: float
      Value: sum(source.close) / len(source.close)
    - Name: volume_avg
      Type: float
      Value: sum(source.volume) / len(source.volume)
    - Name:

**Window DTC**

We'll use a very rough criteria to remove outliers: our model will only work when closing price changes less than a 4%:

```yaml
Anchor:
  Condition: nyse.stats.volatility < 0.04
```

We're using [moving averages](https://www.investopedia.com/terms/m/movingaverage.asp) to generate features based on historical data about a stock:

```yaml
- Type: Blurr:Aggregate:Window
    Name: last_7
    WindowType: count
    WindowValue: -7
    Source: nyse.stats
    Fields:
    - Name: close_avg
      Type: float
      Value: sum(source.close) / len(source.close)
```


## Transforming Data



In [6]:
from blurr_util import print_head, validate, transform

validate('nyse-streaming-dtc.yml')
validate('nyse-window-dtc.yml')

Running syntax validation on nyse-streaming-dtc.yml
Document is valid
Running syntax validation on nyse-window-dtc.yml
Document is valid


Let's run our Streaming DTC for informational purposes only, so we can preview the result of the transformation:

In [7]:
transform(log_files=["./prices-split-adjusted.json"],
          stream_dtc='./nyse-streaming-dtc.yml',
          output_file="./nyse-streaming-dtc-out.log")

In [8]:
print_head("./nyse-streaming-dtc-out.log")

In [9]:
transform(log_files=["./prices-split-adjusted.json"],
          stream_dtc='./nyse-streaming-dtc.yml',
          window_dtc='./nyse-window-dtc.yml',
          output_file="./nyse-processed-data.csv")

Let's now preview the data that will be used to **train our model**

In [10]:
window_out = pd.read_csv("./nyse-processed-data.csv")
window_out.head()

,close._identity,close.value,last._identity,last.close,last.volatility,last.volume,last_3._identity,last_3.close_avg,last_3.max_volatility,last_3.min_volatility,last_3.volatility_avg,last_3.volume_avg,last_7._identity,last_7.close_avg,last_7.max_volatility,last_7.min_volatility,last_7.volatility_avg,last_7.volume_avg
0,AAPL,30.092857,AAPL,29.674286,0.016229,148614900.0,AAPL,29.990953,0.021828,0.014063,0.017373,1.253583e+08,AAPL,30.198979,0.021828,0.009982,0.015492,129615200.0
1,AAPL,29.918571,AAPL,30.092857,0.033464,151473000.0,AAPL,29.927619,0.033464,0.016229,0.023840,1.385484e+08,AAPL,30.130408,0.033464,0.010973,0.018847,133621000.0
2,AAPL,29.418571,AAPL,29.918571,0.006889,108223500.0,AAPL,29.895238,0.033464,0.006889,0.018861,1.361038e+08,AAPL,30.029388,0.033464,0.006889,0.018263,127584900.0
3,AAPL,30.719999,AAPL,29.418571,0.027833,148516900.0,AAPL,29.810000,0.033464,0.006889,0.022729,1.360711e+08,AAPL,29.926531,0.033464,0.006889,0.019203,129081600.0
4,AAPL,30.247143,AAPL,30.719999,0.038361,182501900.0,AAPL,30.019047,0.038361,0.006889,0.024361,1.464141e+08,AAPL,30.017551,0.038361,0.006889,0.022667,138112900.0


## Modelling

**Blurr** is about Data Preparation and Feature Engineering. Modeling is included here for illustration purpose, and the reader can use any modeling library or tool for such purpose.

Let's start by importing the output of our Window DTC as the source dataset. We're dropping unnecessary `_identity` columns:

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def import_dataset():
    data = pd.read_csv("./nyse-processed-data.csv")
    data["close"] = data["close.value"] # Moving close to the last column
    data.drop(['close.value'], 1, inplace=True) 
    data.drop(['close._identity'], 1, inplace=True) 
    data.drop(['last._identity'], 1, inplace=True) 
    data.drop(['last_3._identity'], 1, inplace=True) 
    data.drop(['last_7._identity'], 1, inplace=True) 
    return data

dataset = import_dataset()
dataset.head()

,last.close,last.volatility,last.volume,last_3.close_avg,last_3.max_volatility,last_3.min_volatility,last_3.volatility_avg,last_3.volume_avg,last_7.close_avg,last_7.max_volatility,last_7.min_volatility,last_7.volatility_avg,last_7.volume_avg,close
0,29.674286,0.016229,148614900.0,29.990953,0.021828,0.014063,0.017373,1.253583e+08,30.198979,0.021828,0.009982,0.015492,129615200.0,30.092857
1,30.092857,0.033464,151473000.0,29.927619,0.033464,0.016229,0.023840,1.385484e+08,30.130408,0.033464,0.010973,0.018847,133621000.0,29.918571
2,29.918571,0.006889,108223500.0,29.895238,0.033464,0.006889,0.018861,1.361038e+08,30.029388,0.033464,0.006889,0.018263,127584900.0,29.418571
3,29.418571,0.027833,148516900.0,29.810000,0.033464,0.006889,0.022729,1.360711e+08,29.926531,0.033464,0.006889,0.019203,129081600.0,30.719999
4,30.719999,0.038361,182501900.0,30.019047,0.038361,0.006889,0.024361,1.464141e+08,30.017551,0.038361,0.006889,0.022667,138112900.0,30.247143


Each column represents a Feature, except the rightmost column which represents the Output we're trying to predic

In [12]:
feature_count = len(dataset.columns) - 1
print("#features=" + str(feature_count))

#features=13


We're splitting our dataset into Input Variables (`X`) and the Output Variable (`Y`) using pandas' [`iloc` function](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.iloc.html):

In [13]:
X = dataset.iloc[:, 0:feature_count].values
print(X.shape)

(5978, 13)


In [14]:
Y = dataset.iloc[:, feature_count].values
print(Y.shape)

(5978,)


We need to split between train and test datasets for training and evaluation of the model:

In [15]:
from sklearn.model_selection import train_test_split

X_train_raw, X_test_raw, Y_train_raw, Y_test_raw = train_test_split(X, Y, test_size = 0.2)

Finally, we need to  scale our data before training:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)
Y_train = scaler.fit_transform(Y_train_raw)
Y_test = scaler.transform(Y_test_raw)

It's now time to build and train our model:

In [17]:
# Importing the Keras libraries and packages

import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [18]:
#Initializing Neural Network

model = Sequential()
model.add(Dense(units = 36, kernel_initializer = 'uniform', activation = 'relu', input_dim = feature_count))
model.add(Dense(units = 36, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear'))

# Compiling Neural Network
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])

# Fitting our model 
model.fit(X_train, Y_train, batch_size = 512, epochs = 70, validation_split=0.1, verbose=1)

Train on 4303 samples, validate on 479 samples
Epoch 1/70
4303/4303 [==============================] - 0s 79us/step - loss: 0.1033 - acc: 2.3240e-04 - val_loss: 0.1121 - val_acc: 0.0000e+00
Epoch 2/70
4303/4303 [==============================] - 0s 8us/step - loss: 0.0946 - acc: 2.3240e-04 - val_loss: 0.1001 - val_acc: 0.0000e+00
Epoch 3/70
4303/4303 [==============================] - 0s 7us/step - loss: 0.0813 - acc: 2.3240e-04 - val_loss: 0.0820 - val_acc: 0.0000e+00
Epoch 4/70
4303/4303 [==============================] - 0s 6us/step - loss: 0.0639 - acc: 2.3240e-04 - val_loss: 0.0622 - val_acc: 0.0000e+00
Epoch 5/70
4303/4303 [==============================] - 0s 6us/step - loss: 0.0502 - acc: 2.3240e-04 - val_loss: 0.0491 - val_acc: 0.0000e+00
Epoch 6/70
4303/4303 [==============================] - 0s 7us/step - loss: 0.0399 - acc: 2.3240e-04 - val_loss: 0.0358 - val_acc: 0.0000e+00
Epoch 7/70
4303/4303 [==============================] - 0s 7us/step - loss: 0.0267 - acc: 4.6479e-04

4303/4303 [==============================] - 0s 8us/step - loss: 6.1213e-05 - acc: 4.6479e-04 - val_loss: 5.6451e-05 - val_acc: 0.0000e+00
Epoch 56/70
4303/4303 [==============================] - 0s 8us/step - loss: 5.8882e-05 - acc: 4.6479e-04 - val_loss: 5.3443e-05 - val_acc: 0.0000e+00
Epoch 57/70
4303/4303 [==============================] - 0s 8us/step - loss: 5.6409e-05 - acc: 4.6479e-04 - val_loss: 5.1334e-05 - val_acc: 0.0000e+00
Epoch 58/70
4303/4303 [==============================] - 0s 8us/step - loss: 5.4251e-05 - acc: 4.6479e-04 - val_loss: 4.9525e-05 - val_acc: 0.0000e+00
Epoch 59/70
4303/4303 [==============================] - 0s 8us/step - loss: 5.2653e-05 - acc: 4.6479e-04 - val_loss: 4.8090e-05 - val_acc: 0.0000e+00
Epoch 60/70
4303/4303 [==============================] - 0s 8us/step - loss: 5.0981e-05 - acc: 4.6479e-04 - val_loss: 4.6113e-05 - val_acc: 0.0000e+00
Epoch 61/70
4303/4303 [==============================] - 0s 7us/step - loss: 4.9138e-05 - acc: 4.6479e-04 

We can measure the quality of our model using [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) and [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation):

In [19]:
import math

score = model.evaluate(X_test, Y_test, verbose=0)
print('Model Score: %.5f MSE (%.2f RMSE)' % (score[0], math.sqrt(score[0])))

Model Score: 0.00004 MSE (0.01 RMSE)


Finally, let's plot prediction vs actual data:

In [21]:
import matplotlib.pyplot as plt2

prediction = model.predict(X_test)

plt2.plot(prediction, color='red', label='Prediction')
plt2.plot(Y_test,color='blue', label='Actual')
plt2.legend(loc='best')
plt2.show()